In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Reference: https://carbonati.github.io/posts/random-forests-from-scratch/

In [ ]:
data = pd.read_csv('clinical_data_with_img_features.csv')

In [ ]:
#delete rows with invalid values
data = data.loc[~(data == -999).any(axis=1)]

data

,Patient-ID,De-identify Scout Name,age,sex,major_comorbidity,chemo_before_liver_resection,clinrisk_score,clinrisk_stratified,extrahep_disease,steatosis_yesno,...,vital_status_DFS,progression_or_recurrence_liveronly,months_to_liver_DFS_progression,vital_status_liver_DFS,relevant_notes,Liver,Liver Remnant,Hepatic,Portal,Tumor
0,CRLM-CT-1001,001_recurrence_preop,65,2,0,1,1,0,0,0,...,0,0,117.4,0,NaN,2247110,2139054,32503,12698,1379
1,CRLM-CT-1002,002_recurrence_preop,63,1,0,1,2,0,0,0,...,1,1,13.8,1,NaN,1061502,881098,35784,12994,643
2,CRLM-CT-1003,003_recurrence_preop,52,2,0,0,0,0,0,1,...,0,0,94.7,0,NaN,693475,671340,10246,4563,728
3,CRLM-CT-1004,004_recurrence_preop,39,1,1,1,3,1,0,0,...,0,0,92.0,0,NaN,405935,392535,5136,5813,473
4,CRLM-CT-1005,005_recurrence_preop,87,1,1,1,3,1,0,0,...,0,0,24.7,0,NaN,1413327,1043907,15027,10136,52187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,CRLM-CT-1191,320_recurrence_preop,86,2,1,1,2,0,0,0,...,1,1,13.9,1,NaN,260479,81741,5886,5845,2863
190,CRLM-CT-1192,321_recurrence_preop,66,1,1,1,3,1,0,0,...,1,1,36.1,1,NaN,341718,116584,3962,6209,7780
191,CRLM-CT-1193,324_recurrence_preop,66,1,1,1,2,0,0,1,...,1,1,12.7,1,NaN,485915,345576,3913,8741,1456
192,CRLM-CT-1194,326_recurrence_preop,59,2,1,1,2,0,0,0,...,1,1,19.3,1,NaN,328044,112469,2505,4069,28280


In [ ]:
# Select numerical columns
num_cols = [col for col in data.select_dtypes(include=np.number).columns if data[col].nunique() > 2]

# Standardize numerical columns
for col in num_cols:
    col_mean = data[col].mean()
    col_std = data[col].std()
    data[col] = (data[col] - col_mean) / col_std

data

,Patient-ID,De-identify Scout Name,age,sex,major_comorbidity,chemo_before_liver_resection,clinrisk_score,clinrisk_stratified,extrahep_disease,steatosis_yesno,...,vital_status_DFS,progression_or_recurrence_liveronly,months_to_liver_DFS_progression,vital_status_liver_DFS,relevant_notes,Liver,Liver Remnant,Hepatic,Portal,Tumor
0,CRLM-CT-1001,001_recurrence_preop,0.382207,2,0,1,-1.000454,0,0,0,...,0,0,1.575464,0,NaN,1.764822,2.909069,1.120966,-0.051164,-0.342608
1,CRLM-CT-1002,002_recurrence_preop,0.214581,1,0,1,0.018413,0,0,0,...,1,1,-1.010305,1,NaN,0.060276,0.490537,1.359067,-0.020197,-0.351477
2,CRLM-CT-1003,003_recurrence_preop,-0.707360,2,0,0,-2.019321,0,0,1,...,0,0,1.008891,0,NaN,-0.468836,0.087259,-0.494218,-0.902227,-0.350453
3,CRLM-CT-1004,004_recurrence_preop,-1.796927,1,1,1,1.037280,1,0,0,...,0,0,0.941502,0,NaN,-0.882231,-0.448768,-0.865049,-0.771455,-0.353525
4,CRLM-CT-1005,005_recurrence_preop,2.226090,1,1,1,1.037280,1,0,0,...,0,0,-0.738250,0,NaN,0.566094,0.803552,-0.147262,-0.319194,0.269593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,CRLM-CT-1191,320_recurrence_preop,2.142277,2,1,1,0.018413,0,0,0,...,1,1,-1.007809,1,NaN,-1.091353,-1.046297,-0.810622,-0.768107,-0.324727
190,CRLM-CT-1192,321_recurrence_preop,0.466020,1,1,1,1.037280,1,0,0,...,1,1,-0.453715,1,NaN,-0.974556,-0.979308,-0.950246,-0.730026,-0.265481
191,CRLM-CT-1193,324_recurrence_preop,0.466020,1,1,1,0.018413,0,0,1,...,1,1,-1.037760,1,NaN,-0.767244,-0.539051,-0.953802,-0.465135,-0.341681
192,CRLM-CT-1194,326_recurrence_preop,-0.120670,2,1,1,0.018413,0,0,0,...,1,1,-0.873029,1,NaN,-0.994215,-0.987220,-1.055980,-0.953908,-0.018470


In [ ]:
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

In [ ]:
def draw_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    y_bootstrap = y_train[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train[oob_indices]
    return X_bootstrap, y_bootstrap, X_oob, y_oob

def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [ ]:
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])

    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)

    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}

        # split children for continuous variables
        if type(split_point) in [int, float]:
            for i, value in enumerate(X_bootstrap[:,feature_idx]):
                if value <= split_point:
                    left_child['X_bootstrap'].append(X_bootstrap[i])
                    left_child['y_bootstrap'].append(y_bootstrap[i])
                else:
                    right_child['X_bootstrap'].append(X_bootstrap[i])
                    right_child['y_bootstrap'].append(y_bootstrap[i])
        # split children for categoric variables
        else:
            for i, value in enumerate(X_bootstrap[:,feature_idx]):
                if value == split_point:
                    left_child['X_bootstrap'].append(X_bootstrap[i])
                    left_child['y_bootstrap'].append(y_bootstrap[i])
                else:
                    right_child['X_bootstrap'].append(X_bootstrap[i])
                    right_child['y_bootstrap'].append(y_bootstrap[i])

        split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
        if split_info_gain > best_info_gain:
            best_info_gain = split_info_gain
            left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
            right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
            node = {'information_gain': split_info_gain,
                    'left_child': left_child,
                    'right_child': right_child,
                    'split_point': split_point,
                    'feature_idx': feature_idx}

    return node

In [ ]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred


def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [ ]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    return tree_ls

In [ ]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [ ]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

In [ ]:
# remove the '%' symbol from the 'total_response_percent' column
data['total_response_percent'] = data['total_response_percent'].str.rstrip('%')
data['necrosis_percent'] = data['necrosis_percent'].str.rstrip('%')
data['fibrosis_percent'] = data['fibrosis_percent'].str.rstrip('%')
data['mucin_percent'] = data['mucin_percent'].str.rstrip('%')

# convert the 'total_response_percent' column to float
data['total_response_percent'] = data['total_response_percent'].astype(float)
data['necrosis_percent'] = data['necrosis_percent'].astype(float)
data['fibrosis_percent'] = data['fibrosis_percent'].astype(float)
data['mucin_percent'] = data['mucin_percent'].astype(float)

X = data.drop(['Patient-ID', 'De-identify Scout Name', 'progression_or_recurrence_liveronly', 'relevant_notes'], axis=1)
y = data['progression_or_recurrence_liveronly']


In [ ]:
# get the column names of the original data
original_column_names = X.columns

In [ ]:
features = X.columns.tolist()
nb_train = int(np.floor(0.9 * len(data)))
df = X.sample(frac=1, random_state=197)
X_train = data[features][:]

In [ ]:
y_train = data['progression_or_recurrence_liveronly'][:].values
y_train

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
def train_test_split(X, y, test_size=0.25, random_state=None):
    """
    Split arrays or matrices into random train and test subsets.

    Parameters
    ----------
    X : numpy array-like, shape (n_samples, n_features)
        The input data.

    y : numpy array-like, shape (n_samples,)
        The target values.

    test_size : float, optional (default=0.25)
        The proportion of the dataset to include in the test split.

    random_state : int or RandomState, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator.

    Returns
    -------
    X_train : numpy array-like, shape (n_train_samples, n_features)
        The training input samples.

    X_test : numpy array-like, shape (n_test_samples, n_features)
        The test input samples.

    y_train :  numpy array-like, shape (n_train_samples,)
        The training target values.

    y_test : numpy array-like, shape (n_test_samples,)
        The test target values.
    """

    # Set random seed if specified
    if random_state is not None:
        np.random.seed(random_state)

    # Get number of samples
    n_samples = X.shape[0]

    # Shuffle indices
    indices = np.random.permutation(n_samples)

    # Calculate number of test samples
    n_test_samples = int(test_size * n_samples)

    # Get test indices
    test_indices = indices[:n_test_samples]

    # Get train indices
    train_indices = indices[n_test_samples:]

    # Split data into train and test subsets
    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]

    return X_train, X_test, y_train, y_test


In [ ]:
len(X)

166

In [ ]:
model = random_forest(X, y_train, n_estimators=100, max_features=5, max_depth=10, min_samples_split=2)

OOB estimate: 0.46


In [ ]:
n_estimators = 100
max_features = 3
max_depth = 10
min_samples_split = 2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train.values, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

X_train.columns = original_column_names
X_train


,age,sex,major_comorbidity,chemo_before_liver_resection,clinrisk_score,clinrisk_stratified,extrahep_disease,steatosis_yesno,presence_sinusoidal_dilata,NASH_score,...,progression_or_recurrence,months_to_DFS_progression,vital_status_DFS,months_to_liver_DFS_progression,vital_status_liver_DFS,Liver,Liver Remnant,Hepatic,Portal,Tumor
0,-0.874986,1.0,0.0,0.0,-1.000454,0.0,0.0,0.0,0.0,-0.586758,...,0.0,1.361644,0.0,1.078777,0.0,-0.780112,-0.965275,-0.521794,-0.467646,-0.165640
1,1.136523,2.0,1.0,1.0,1.037280,1.0,0.0,0.0,0.0,-0.586758,...,1.0,-0.753567,1.0,-0.947907,1.0,1.385616,0.104954,1.941076,2.076651,1.626910
2,-0.707360,2.0,0.0,0.0,-2.019321,0.0,0.0,1.0,0.0,1.999132,...,0.0,1.291719,0.0,1.008891,0.0,-0.468836,0.087259,-0.494218,-0.902227,-0.350453
3,-0.623547,2.0,1.0,0.0,0.018413,0.0,0.0,1.0,0.0,1.137169,...,0.0,0.657406,0.0,0.374929,0.0,-0.872456,-0.575563,-0.181878,-0.615993,-0.347296
4,-0.120670,1.0,0.0,1.0,0.018413,0.0,0.0,1.0,0.0,1.999132,...,1.0,-0.661167,1.0,0.896575,0.0,0.566413,0.590289,0.848249,1.060292,-0.315859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.382207,2.0,1.0,1.0,0.018413,0.0,1.0,1.0,0.0,0.275205,...,1.0,-0.998302,1.0,-1.279864,1.0,-1.143763,-0.837477,-0.868750,-0.959662,-0.316787
129,-1.377863,2.0,0.0,1.0,2.056147,1.0,0.0,1.0,0.0,0.275205,...,1.0,-0.895913,1.0,-1.177531,1.0,-0.748364,-0.719930,-0.700896,-0.417534,0.023883
130,-0.791173,2.0,0.0,0.0,0.018413,0.0,0.0,0.0,0.0,-0.586758,...,1.0,0.120487,1.0,0.836673,1.0,-1.050078,-0.687163,-0.904817,-0.724063,-0.347790
131,-0.455922,1.0,0.0,1.0,-1.000454,0.0,0.0,0.0,0.0,-0.586758,...,0.0,-0.121751,0.0,-0.403797,0.0,0.045042,0.491312,-0.685221,0.061509,-0.311063


In [ ]:
y_test

array([0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0])

In [ ]:
preds = predict_rf(model, pd.DataFrame(X_test)).flatten()
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))
print(np.mean(acc))

Testing accuracy: 0.848
0.8484848484848485


Reference: https://github.com/sethbilliau/FeatureImportance/blob/main/permutationFeatureImportance.ipynb

In [ ]:
pd.DataFrame(X_test).columns

RangeIndex(start=0, stop=29, step=1)

In [ ]:
def calculate_feature_importance(X, y, model, column_names):
    """
    Calculate the feature importance for a random forest model without using sklearn.
    
    Parameters:
    X (numpy.ndarray): The input feature matrix.
    y (numpy.ndarray): The target variable array.
    model (numpy.ndarray): The trained random forest model.
    column_names (list): The names of columns
    
    Returns:
    pandas.DataFrame: A dataframe containing the feature importance values sorted in descending order.
    """
    
    # Get the number of features
    n_features = X.shape[1]
    
    # Initialize an empty array to hold the importance scores
    importance = np.zeros(n_features)
    
    # Calculate the importance of each feature
    for i in range(n_features):
        # Get the predictions for the original data
        y_pred = predict_rf(model, pd.DataFrame(X))

        # Shuffle the values of the ith feature
        X_shuffled = X.copy()
        np.random.shuffle(X_shuffled[:, i])

        # Get the predictions for the shuffled data
        y_pred_shuffled = predict_rf(model, pd.DataFrame(X_shuffled))

        # Calculate the importance score as the difference in accuracy
        importance[i] = np.mean((y_pred - y_pred_shuffled) ** 2)

    # Normalize the importance scores
    importance = importance / np.sum(importance)

    # Create a pandas dataframe with the importance scores and feature names
    feature_names = [column_names[i] for i in range(n_features)]
    importance_df = pd.DataFrame({"importance": importance, "feature": feature_names})

    # Sort the dataframe in descending order of importance
    importance_df = importance_df.sort_values("importance", ascending=False).reset_index(drop=True)

    return importance_df
